In [1]:
from distributed import Client
client = Client()
# import pandas as pd
import modin.pandas as pd
import os
import json
from pathlib import Path
from urllib.request import urlretrieve
from tqdm import tqdm
from zipfile import ZipFile
pd.set_option("display.max_columns", None)
print("Setup Complete")

2023-03-18 07:30:37,689 - distributed.diskutils - INFO - Found stale lock file and directory '/var/folders/h2/6110hgtx6b7400ybw7yjhn400000gn/T/dask-worker-space/worker-drpi8364', purging
2023-03-18 07:30:37,690 - distributed.diskutils - INFO - Found stale lock file and directory '/var/folders/h2/6110hgtx6b7400ybw7yjhn400000gn/T/dask-worker-space/worker-9bnhtp9w', purging
2023-03-18 07:30:37,691 - distributed.diskutils - INFO - Found stale lock file and directory '/var/folders/h2/6110hgtx6b7400ybw7yjhn400000gn/T/dask-worker-space/worker-yflpwl7y', purging
2023-03-18 07:30:37,692 - distributed.diskutils - INFO - Found stale lock file and directory '/var/folders/h2/6110hgtx6b7400ybw7yjhn400000gn/T/dask-worker-space/worker-4j800s2z', purging
2023-03-18 07:30:37,693 - distributed.diskutils - INFO - Found stale lock file and directory '/var/folders/h2/6110hgtx6b7400ybw7yjhn400000gn/T/dask-worker-space/worker-rtxd301x', purging
2023-03-18 07:30:37,693 - distributed.diskutils - INFO - Found st

Setup Complete


In [ ]:
# ETL Pipes for Capstone Project

# fetch data
# read_df
# write to local, include to convert json to parquet file for smaller files
# upload to GCS
# then upload to Bigquery


In [5]:
# Define download progress hook
def download_progress_hook(block_num, block_size, total_size):
    global progress_bar
    if not progress_bar:
        progress_bar = tqdm(total=total_size, unit='B', unit_scale=True)
    downloaded = block_num * block_size
    progress_bar.update(downloaded - progress_bar.n)
    if downloaded >= total_size:
        progress_bar = None


In [9]:
dataset_url = "https://www.dropbox.com/s/a1kl5e35j4o53mz/bandcamp-items-json.zip?dl=1"
dataset_url.split("/")[-1].split("?")[0]

'bandcamp-items-json.zip'

In [7]:

# dataset_url = "https://www.dropbox.com/s/a1kl5e35j4o53mz/bandcamp-items-json.zip?dl=1"
dataset_url_2 = "https://www.dropbox.com/s/wd38q80el16i19q/1000000-bandcamp-sales.zip?dl=1"
# This is global parameter
progress_bar = None 

# dataset_local = "/Users/reneboygarcia/Downloads/bandcamp-items-json.zip"
def extract_data(dataset_url: str):
    # Download file
    folder_name = dataset_url.split("/")[-1].split("?")[0]
    print(f"Downloading {folder_name}")
    folder_file, _ = urlretrieve(dataset_url, folder_name, reporthook=download_progress_hook)
    zip_file = ZipFile(folder_name)
    unzip_folder_file = zip_file.extractall()
    print(f"Download Complete..extracted zip file")
    return unzip_folder_file

json_folder = extract_data(dataset_url_2)

80.6MB [00:32, 2.46MB/s]                            


Download Complete..extracted zip file


In [ ]:
file_name = "albums-full-info-1"
# Read and Tweak DataFrame
def read_df(src: str) -> pd.DataFrame:
    with open(json_files[0]) as j:
        data = json.load(j)
        df = pd.read_json(data)
        df = pd.json_normalize(df.to_dict("records"), sep="_")
        

In [38]:
print(os.listdir(os.getcwd()))

['1000000-bandcamp-sales.csv', '1000000-bandcamp-sales.zip', '_playground.ipynb', 'bandcamp-items-json.zip']


In [35]:
# Loop over the JSON files in the extracted directory
json_files = [f for f in os.listdir(os.getcwd()) if f.endswith('.csv')]
json_files

['1000000-bandcamp-sales.csv']

In [42]:
# Loop over the JSON files in the extracted directory
csv_file = [f for f in os.listdir(os.getcwd()) if f.endswith('.csv')]
parent_dir = os.getcwd()
file_path = os.path.join(parent_dir, csv_file[0])
df = pd.read_csv(file_path, index_col=[0])
df


# with open(json_files[0]) as j:
#     data = json.load(j)
#     df = pd.read_json(data)
#     df = pd.json_normalize(df.to_dict("records"), sep="_")

# df.head()

,_id,art_url,item_type,utc_date,country_code,track_album_slug_text,country,slug_type,amount_paid_fmt,item_price,item_description,url,amount_paid,releases,artist_name,currency,album_title,amount_paid_usd,package_image_id,amount_over_fmt,item_slug,addl_count
0,1599688803.5175&//girlbanddublin.bandcamp.com/...,https://f4.bcbits.com/img/a0206405257_7.jpg,a,1.599689e+09,gb,NaN,United Kingdom,a,$9.99,9.990000,Live at Vicar Street,//girlbanddublin.bandcamp.com/album/live-at-vi...,9.99,NaN,Girl Band,USD,NaN,9.99,NaN,NaN,NaN,NaN
1,1599688805.27838&//maharettarecords.bandcamp.c...,https://f4.bcbits.com/img/a2984241552_7.jpg,a,1.599689e+09,fi,NaN,Finland,a,£1,1.000000,Neurogen,//maharettarecords.bandcamp.com/album/neurogen,1.00,NaN,Jirah,GBP,NaN,1.30,NaN,NaN,NaN,NaN
2,1599688805.90646&//maharettarecords.bandcamp.c...,https://f4.bcbits.com/img/a3320494770_7.jpg,a,1.599689e+09,fi,NaN,Finland,a,£3,3.000000,The Last Snare Bender,//maharettarecords.bandcamp.com/album/the-last...,3.00,NaN,D-Ther,GBP,NaN,3.90,NaN,NaN,NaN,NaN
3,1599688806.94234&//alicesitski.bandcamp.com/al...,https://f4.bcbits.com/img/0020476345_37.jpg,p,1.599689e+09,gb,NaN,United Kingdom,a,€10.50,10.500000,Limited Edition Compact Disc,//alicesitski.bandcamp.com/album/white-noise-tv,10.50,NaN,WHITE NOISE TV,EUR,WHITE NOISE TV,12.39,20476345.0,NaN,NaN,NaN
4,1599688809.07942&//linguaignota.bandcamp.com/t...,https://f4.bcbits.com/img/a3428873396_7.jpg,t,1.599689e+09,us,NaN,United States,t,$1,1.000000,O Ruthless Great Divine Director,//linguaignota.bandcamp.com/track/o-ruthless-g...,1.00,NaN,LINGUA IGNOTA,USD,NaN,1.00,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,1601647409.59517&//eternalchampion.bandcamp.co...,https://f4.bcbits.com/img/0021839465_37.jpg,p,1.601647e+09,ca,NaN,Canada,a,$20,20.000000,Black vinyl,//eternalchampion.bandcamp.com/album/ravening-...,20.00,NaN,Eternal Champion,USD,RAVENING IRON,20.00,21839465.0,NaN,NaN,NaN
999996,1601647409.61911&//goodmusic2020.bandcamp.com/...,https://f4.bcbits.com/img/a3708391191_7.jpg,a,1.601647e+09,us,NaN,United States,a,$20.20,20.200001,Good Music to Avert the Collapse of American D...,//goodmusic2020.bandcamp.com/album/good-music-...,20.20,NaN,Various Artists Working to Avert the Collapse ...,USD,NaN,20.20,NaN,NaN,NaN,NaN
999997,1601647409.63413&//geometriclullaby.bandcamp.c...,https://f4.bcbits.com/img/a0358085735_7.jpg,a,1.601647e+09,au,NaN,Australia,a,$1,0.000000,O'discordia,//geometriclullaby.bandcamp.com/album/odiscordia,1.00,NaN,w i n t e r q u i l t,USD,NaN,1.00,NaN,NaN,NaN,NaN
999998,1601647409.77243&//bewitcher.bandcamp.com/merc...,https://f4.bcbits.com/img/0019759485_37.jpg,p,1.601647e+09,us,NaN,United States,p,$6,6.000000,Bewitcher Logo Patch,//bewitcher.bandcamp.com/merch/bewitcher-logo-...,6.00,NaN,Bewitcher,USD,NaN,6.00,19759485.0,NaN,NaN,NaN


In [ ]:
import os
import json
import zipfile
import urllib.request
os.environ["MODIN_ENGINE"] = "ray" 
import modin.pandas as pd
from distributed import Client

client = Client()
dataset_url = "https://www.dropbox.com/s/a1kl5e35j4o53mz/bandcamp-items-json.zip?dl=1"

# Download and extract the zip file
urllib.request.urlretrieve(dataset_url, "bandcamp-items-json.zip")
zip_file = zipfile.ZipFile("bandcamp-items-json.zip")
zip_file.extractall()

# Loop over the JSON files in the extracted directory
json_files = [f for f in os.listdir() if f.endswith('.json')]
for json_file in json_files:
    with open(json_file) as j:
        data = json.load(j)
        df = pd.read_json(data)
        df = pd.json_normalize(df.to_dict("records"), sep="_")
        print(df.head())


In [5]:
list(df.columns)

['_id',
 '@id',
 'datePublished',
 'url',
 'dateModified',
 'duration',
 'isrcCode',
 '@type',
 'duration_secs',
 'name',
 'image',
 '@context',
 'keywords',
 'inAlbum',
 'numTracks',
 'description',
 'albumRelease',
 'track',
 'comment',
 'offers_priceSpecification_minPrice',
 'offers_url',
 'offers_price',
 'offers_availability',
 'offers_priceCurrency',
 'offers_@type',
 'byArtist_sameAs',
 'byArtist_description',
 'byArtist_@id',
 'byArtist_genre',
 'byArtist_@type',
 'byArtist_image',
 'byArtist_name',
 'recordingOf_lyrics_text',
 'recordingOf_lyrics_@type',
 'recordingOf_@type',
 'recordingOf',
 'inAlbum_@id',
 'inAlbum_@type',
 'inAlbum_name',
 'offers',
 'track_numberOfItems',
 'track_@type',
 'track_itemListElement']

In [6]:
df.info()

<class 'modin.pandas.dataframe.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 43 columns):
 #   Column                              Non-Null Count   Dtype  
---  ----------------------------------  ---------------  -----  
 0   _id                                 100000 non-null  object
 1   @id                                 100000 non-null  object
 2   datePublished                       100000 non-null  object
 3   url                                 50511 non-null   object
 4   dateModified                        100000 non-null  object
 5   duration                            50511 non-null   object
 6   isrcCode                            11622 non-null   object
 7   @type                               100000 non-null  object
 8   duration_secs                       50511 non-null   float64
 9   name                                100000 non-null  object
 10  image                               99981 non-null   object
 11  @context                        

2023-03-17 19:36:55,276 - distributed.worker - ERROR - Exception during execution of task lambda-27d2510e280ed31a18b4e8df147eab8b.
Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/distributed/worker.py", line 2366, in _prepare_args_for_execution
    data[k] = self.data[k]
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/distributed/spill.py", line 257, in __getitem__
    return super().__getitem__(key)
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/zict/buffer.py", line 108, in __getitem__
    raise KeyError(key)
KeyError: 'deploy_dask_func-5db2d349-8b7e-48b5-8a59-c1d47f7397d6'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/distributed/worker.py", line 2247, in execute
    args2, kwargs2 = self._prepare_args_for_execution(ts, args, kwargs)
  File